In [1]:
import os
import logging
import time
import numpy as np
import boto3
from datetime import datetime
import pandas as pd
import pickle

In [2]:
# The difference between UTC and local timezone
timezone_offset = 0

This notebook is for measuring the function execution delay and scheduling overhead of AWS Lambda and AWS Step Functions.

# Execute the Application

In [3]:
sfn_client = boto3.client('stepfunctions')

In [4]:
stateMachineArn='arn:aws:states:us-east-2:499537426559:stateMachine:AppSFNDelay'

The application is composed of 4 serverless functions that form a sequence.
Workflow: f1 -> f2 -> f3 -> f4
f1: sleep for 100 ms
f2: sleep for 200 ms
f3: sleep for 300 ms
f4: sleep for 400 ms

## Test Run

In [5]:
sfn_client.start_execution(
    stateMachineArn=stateMachineArn,
    input="{\"para\":96}"
)

{'executionArn': 'arn:aws:states:us-east-2:499537426559:express:AppSFNDelay:a26c4c6e-cbe1-4f04-bc57-7df6860770be:1d12985d-d85d-4045-852d-3d12d518bc25',
 'startDate': datetime.datetime(2021, 12, 28, 23, 34, 0, 130000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '5cab34b7-1332-4300-a31b-8b5b9923443c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5cab34b7-1332-4300-a31b-8b5b9923443c',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '178'},
  'RetryAttempts': 0}}

## Configure Logging

In [6]:
logging.basicConfig(filename='AppExecution.log', encoding='utf-8', format='%(asctime)s.%(msecs)03d %(message)s', datefmt='%Y-%m-%d %H:%M:%S', level=logging.INFO)

## Execute AppSFNDelay

In [7]:
for i in range(5000):
    payload_str="{\"para\":0}"
    response = sfn_client.start_execution(stateMachineArn=stateMachineArn, input=payload_str)
    RequestId = response.get('ResponseMetadata', {}).get('RequestId')
    StatusCode = response.get('ResponseMetadata', {}).get('HTTPStatusCode', 'ERR')
    logging.info(f'{i+1} {StatusCode} {RequestId}')
    time.sleep(4)

## Get the start time and the end time

In [3]:
app_exeuction_start_time = ' '.join(os.popen('head -1 AppExecution.log').read().split(' ')[:2])
app_execution_end_time = ' '.join(os.popen('tail -1 AppExecution.log').read().split(' ')[:2])
app_exeuction_start_time = datetime.strptime(app_exeuction_start_time, '%Y-%m-%d %H:%M:%S.%f')
app_execution_end_time = datetime.strptime(app_execution_end_time, '%Y-%m-%d %H:%M:%S.%f')

In [4]:
app_exeuction_start_time

datetime.datetime(2021, 12, 28, 23, 36, 29, 543000)

In [5]:
app_execution_end_time

datetime.datetime(2021, 12, 29, 5, 20, 58, 923000)

In [6]:
app_exeuction_start_time = int(datetime.timestamp(app_exeuction_start_time)) -10

In [7]:
app_execution_end_time = int(datetime.timestamp(app_execution_end_time)) + 20

# Retrieve Logs

In [8]:
logclient = boto3.client('logs')

## Query Step Functions Logs for App Execution Time

In [40]:
query_sfn_AppSFNDelay = logclient.start_query(
    logGroupName='/aws/vendedlogs/states/{}-Logs'.format('AppSFNDelay'),
    queryString="fields id, type, @timestamp| filter id = 1 | sort id desc",
    startTime=app_exeuction_start_time,
    endTime=app_execution_end_time,
    limit = 10000
)

In [43]:
query_results_sfn_AppSFNDelay_execution_start = logclient.get_query_results(
    queryId=query_sfn_AppSFNDelay['queryId']
)

In [44]:
query_sfn_AppSFNDelay = logclient.start_query(
    logGroupName='/aws/vendedlogs/states/{}-Logs'.format('AppSFNDelay'),
    queryString="fields id, type, @timestamp| filter id = 22 | sort id desc",
    startTime=app_exeuction_start_time,
    endTime=app_execution_end_time,
    limit = 10000
)

In [45]:
query_results_sfn_AppSFNDelay_execution_end = logclient.get_query_results(
    queryId=query_sfn_AppSFNDelay['queryId']
)

In [53]:
AppSFNDelay_starttimestamp = np.sort([datetime.timestamp(datetime.strptime(item[2]['value'], '%Y-%m-%d %H:%M:%S.%f'))+timezone_offset*3600 for item in query_results_sfn_AppSFNDelay_execution_start['results'] if item[1]['value']=='ExecutionStarted'])
AppSFNDelay_endtimestamp = np.sort([datetime.timestamp(datetime.strptime(item[2]['value'], '%Y-%m-%d %H:%M:%S.%f'))+timezone_offset*3600 for item in query_results_sfn_AppSFNDelay_execution_end['results'] if item[1]['value']=='ExecutionSucceeded'])

In [54]:
pd.DataFrame({'Start': AppSFNDelay_starttimestamp, 'End':AppSFNDelay_endtimestamp}).to_csv('AppSFNDelay_SFN_Logs.csv', index=False)

In [55]:
AppSFNDelay_sfn_logs = pd.read_csv('AppSFNDelay_SFN_Logs.csv', low_memory=False)

In [56]:
AppSFNDelay_sfn_logs.shape

(5000, 2)

## Query Step Functions Logs for Function Execution Time

### f1

In [110]:
query_sfn_AppSFNDelay_f1_scheduled = logclient.start_query(
    logGroupName='/aws/vendedlogs/states/{}-Logs'.format('AppSFNDelay'),
    queryString="fields id, type, @timestamp| filter id = 3 | sort id desc",
    startTime=app_exeuction_start_time,
    endTime=app_execution_end_time,
    limit = 10000
)
time.sleep(1)
query_sfn_AppSFNDelay_f1_start = logclient.start_query(
    logGroupName='/aws/vendedlogs/states/{}-Logs'.format('AppSFNDelay'),
    queryString="fields id, type, @timestamp| filter id = 4 | sort id desc",
    startTime=app_exeuction_start_time,
    endTime=app_execution_end_time,
    limit = 10000
)
time.sleep(1)
query_sfn_AppSFNDelay_f1_end = logclient.start_query(
    logGroupName='/aws/vendedlogs/states/{}-Logs'.format('AppSFNDelay'),
    queryString="fields id, type, @timestamp| filter id = 5 | sort id desc",
    startTime=app_exeuction_start_time,
    endTime=app_execution_end_time,
    limit = 10000
)
time.sleep(3)
query_results_sfn_AppSFNDelay_f1_scheduled = logclient.get_query_results(
    queryId=query_sfn_AppSFNDelay_f1_scheduled['queryId']
)
query_results_sfn_AppSFNDelay_f1_start = logclient.get_query_results(
    queryId=query_sfn_AppSFNDelay_f1_start['queryId']
)
query_results_sfn_AppSFNDelay_f1_end = logclient.get_query_results(
    queryId=query_sfn_AppSFNDelay_f1_end['queryId']
)
AppSFNDelay_f1_scheduled_timestamp = np.sort([datetime.timestamp(datetime.strptime(item[2]['value'], '%Y-%m-%d %H:%M:%S.%f'))+timezone_offset*3600 for item in query_results_sfn_AppSFNDelay_f1_scheduled['results'] if item[1]['value']=='TaskScheduled'])
AppSFNDelay_f1_start_timestamp = np.sort([datetime.timestamp(datetime.strptime(item[2]['value'], '%Y-%m-%d %H:%M:%S.%f'))+timezone_offset*3600 for item in query_results_sfn_AppSFNDelay_f1_start['results'] if item[1]['value']=='TaskStarted'])
AppSFNDelay_f1_end_timestamp = np.sort([datetime.timestamp(datetime.strptime(item[2]['value'], '%Y-%m-%d %H:%M:%S.%f'))+timezone_offset*3600 for item in query_results_sfn_AppSFNDelay_f1_end['results'] if item[1]['value']=='TaskSucceeded'])
pd.DataFrame({'Scheduled': AppSFNDelay_f1_scheduled_timestamp, 'Start': AppSFNDelay_f1_start_timestamp, 'End':AppSFNDelay_f1_end_timestamp}).to_csv('AppSFNDelay_SFN_f1_Logs.csv', index=False)
AppSFNDelay_sfn_f1_logs = pd.read_csv('AppSFNDelay_SFN_f1_Logs.csv', low_memory=False)
AppSFNDelay_sfn_f1_logs.shape

(5000, 3)

### f2

In [130]:
query_sfn_AppSFNDelay_f2_scheduled = logclient.start_query(
    logGroupName='/aws/vendedlogs/states/{}-Logs'.format('AppSFNDelay'),
    queryString="fields id, type, @timestamp| filter id = 8 | sort id desc",
    startTime=app_exeuction_start_time,
    endTime=app_execution_end_time,
    limit = 10000
)
time.sleep(1)
query_sfn_AppSFNDelay_f2_start = logclient.start_query(
    logGroupName='/aws/vendedlogs/states/{}-Logs'.format('AppSFNDelay'),
    queryString="fields id, type, @timestamp| filter id = 9 | sort id desc",
    startTime=app_exeuction_start_time,
    endTime=app_execution_end_time,
    limit = 10000
)
time.sleep(1)
query_sfn_AppSFNDelay_f2_end = logclient.start_query(
    logGroupName='/aws/vendedlogs/states/{}-Logs'.format('AppSFNDelay'),
    queryString="fields id, type, @timestamp| filter id = 10 | sort id desc",
    startTime=app_exeuction_start_time,
    endTime=app_execution_end_time,
    limit = 10000
)
time.sleep(3)
query_results_sfn_AppSFNDelay_f2_scheduled = logclient.get_query_results(
    queryId=query_sfn_AppSFNDelay_f2_scheduled['queryId']
)
query_results_sfn_AppSFNDelay_f2_start = logclient.get_query_results(
    queryId=query_sfn_AppSFNDelay_f2_start['queryId']
)
query_results_sfn_AppSFNDelay_f2_end = logclient.get_query_results(
    queryId=query_sfn_AppSFNDelay_f2_end['queryId']
)
AppSFNDelay_f2_scheduled_timestamp = np.sort([datetime.timestamp(datetime.strptime(item[2]['value'], '%Y-%m-%d %H:%M:%S.%f'))+timezone_offset*3600 for item in query_results_sfn_AppSFNDelay_f2_scheduled['results'] if item[1]['value']=='TaskScheduled'])
AppSFNDelay_f2_start_timestamp = np.sort([datetime.timestamp(datetime.strptime(item[2]['value'], '%Y-%m-%d %H:%M:%S.%f'))+timezone_offset*3600 for item in query_results_sfn_AppSFNDelay_f2_start['results'] if item[1]['value']=='TaskStarted'])
AppSFNDelay_f2_end_timestamp = np.sort([datetime.timestamp(datetime.strptime(item[2]['value'], '%Y-%m-%d %H:%M:%S.%f'))+timezone_offset*3600 for item in query_results_sfn_AppSFNDelay_f2_end['results'] if item[1]['value']=='TaskSucceeded'])
pd.DataFrame({'Scheduled': AppSFNDelay_f2_scheduled_timestamp, 'Start': AppSFNDelay_f2_start_timestamp, 'End':AppSFNDelay_f2_end_timestamp}).to_csv('AppSFNDelay_SFN_f2_Logs.csv', index=False)
AppSFNDelay_sfn_f2_logs = pd.read_csv('AppSFNDelay_SFN_f2_Logs.csv', low_memory=False)
AppSFNDelay_sfn_f2_logs.shape

(5000, 3)

### f3

In [131]:
query_sfn_AppSFNDelay_f3_scheduled = logclient.start_query(
    logGroupName='/aws/vendedlogs/states/{}-Logs'.format('AppSFNDelay'),
    queryString="fields id, type, @timestamp| filter id = 13 | sort id desc",
    startTime=app_exeuction_start_time,
    endTime=app_execution_end_time,
    limit = 10000
)
time.sleep(1)
query_sfn_AppSFNDelay_f3_start = logclient.start_query(
    logGroupName='/aws/vendedlogs/states/{}-Logs'.format('AppSFNDelay'),
    queryString="fields id, type, @timestamp| filter id = 14 | sort id desc",
    startTime=app_exeuction_start_time,
    endTime=app_execution_end_time,
    limit = 10000
)
time.sleep(1)
query_sfn_AppSFNDelay_f3_end = logclient.start_query(
    logGroupName='/aws/vendedlogs/states/{}-Logs'.format('AppSFNDelay'),
    queryString="fields id, type, @timestamp| filter id = 15 | sort id desc",
    startTime=app_exeuction_start_time,
    endTime=app_execution_end_time,
    limit = 10000
)
time.sleep(3)
query_results_sfn_AppSFNDelay_f3_scheduled = logclient.get_query_results(
    queryId=query_sfn_AppSFNDelay_f3_scheduled['queryId']
)
query_results_sfn_AppSFNDelay_f3_start = logclient.get_query_results(
    queryId=query_sfn_AppSFNDelay_f3_start['queryId']
)
query_results_sfn_AppSFNDelay_f3_end = logclient.get_query_results(
    queryId=query_sfn_AppSFNDelay_f3_end['queryId']
)
AppSFNDelay_f3_scheduled_timestamp = np.sort([datetime.timestamp(datetime.strptime(item[2]['value'], '%Y-%m-%d %H:%M:%S.%f'))+timezone_offset*3600 for item in query_results_sfn_AppSFNDelay_f3_scheduled['results'] if item[1]['value']=='TaskScheduled'])
AppSFNDelay_f3_start_timestamp = np.sort([datetime.timestamp(datetime.strptime(item[2]['value'], '%Y-%m-%d %H:%M:%S.%f'))+timezone_offset*3600 for item in query_results_sfn_AppSFNDelay_f3_start['results'] if item[1]['value']=='TaskStarted'])
AppSFNDelay_f3_end_timestamp = np.sort([datetime.timestamp(datetime.strptime(item[2]['value'], '%Y-%m-%d %H:%M:%S.%f'))+timezone_offset*3600 for item in query_results_sfn_AppSFNDelay_f3_end['results'] if item[1]['value']=='TaskSucceeded'])
pd.DataFrame({'Scheduled': AppSFNDelay_f3_scheduled_timestamp, 'Start': AppSFNDelay_f3_start_timestamp, 'End':AppSFNDelay_f3_end_timestamp}).to_csv('AppSFNDelay_SFN_f3_Logs.csv', index=False)
AppSFNDelay_sfn_f3_logs = pd.read_csv('AppSFNDelay_SFN_f3_Logs.csv', low_memory=False)
AppSFNDelay_sfn_f3_logs.shape

(5000, 3)

### f4

In [132]:
query_sfn_AppSFNDelay_f4_scheduled = logclient.start_query(
    logGroupName='/aws/vendedlogs/states/{}-Logs'.format('AppSFNDelay'),
    queryString="fields id, type, @timestamp| filter id = 18 | sort id desc",
    startTime=app_exeuction_start_time,
    endTime=app_execution_end_time,
    limit = 10000
)
time.sleep(1)
query_sfn_AppSFNDelay_f4_start = logclient.start_query(
    logGroupName='/aws/vendedlogs/states/{}-Logs'.format('AppSFNDelay'),
    queryString="fields id, type, @timestamp| filter id = 19 | sort id desc",
    startTime=app_exeuction_start_time,
    endTime=app_execution_end_time,
    limit = 10000
)
time.sleep(1)
query_sfn_AppSFNDelay_f4_end = logclient.start_query(
    logGroupName='/aws/vendedlogs/states/{}-Logs'.format('AppSFNDelay'),
    queryString="fields id, type, @timestamp| filter id = 20 | sort id desc",
    startTime=app_exeuction_start_time,
    endTime=app_execution_end_time,
    limit = 10000
)
time.sleep(3)
query_results_sfn_AppSFNDelay_f4_scheduled = logclient.get_query_results(
    queryId=query_sfn_AppSFNDelay_f4_scheduled['queryId']
)
query_results_sfn_AppSFNDelay_f4_start = logclient.get_query_results(
    queryId=query_sfn_AppSFNDelay_f4_start['queryId']
)
query_results_sfn_AppSFNDelay_f4_end = logclient.get_query_results(
    queryId=query_sfn_AppSFNDelay_f4_end['queryId']
)
AppSFNDelay_f4_scheduled_timestamp = np.sort([datetime.timestamp(datetime.strptime(item[2]['value'], '%Y-%m-%d %H:%M:%S.%f'))+timezone_offset*3600 for item in query_results_sfn_AppSFNDelay_f4_scheduled['results'] if item[1]['value']=='TaskScheduled'])
AppSFNDelay_f4_start_timestamp = np.sort([datetime.timestamp(datetime.strptime(item[2]['value'], '%Y-%m-%d %H:%M:%S.%f'))+timezone_offset*3600 for item in query_results_sfn_AppSFNDelay_f4_start['results'] if item[1]['value']=='TaskStarted'])
AppSFNDelay_f4_end_timestamp = np.sort([datetime.timestamp(datetime.strptime(item[2]['value'], '%Y-%m-%d %H:%M:%S.%f'))+timezone_offset*3600 for item in query_results_sfn_AppSFNDelay_f4_end['results'] if item[1]['value']=='TaskSucceeded'])
pd.DataFrame({'Scheduled': AppSFNDelay_f4_scheduled_timestamp, 'Start': AppSFNDelay_f4_start_timestamp, 'End':AppSFNDelay_f4_end_timestamp}).to_csv('AppSFNDelay_SFN_f4_Logs.csv', index=False)
AppSFNDelay_sfn_f4_logs = pd.read_csv('AppSFNDelay_SFN_f4_Logs.csv', low_memory=False)
AppSFNDelay_sfn_f4_logs.shape

(5000, 3)

## Query Lambda Logs for Function Execution Time

In [116]:
def lambda_report_log_to_dict(log):
    res={}
    lis=[item.split(': ') for item in log[1]['value'].split('\t')]
    res['RequestId']=lis[0][1]
    res['Duration']=float(lis[1][1].split(' ')[0])
    res['Billed_Duration']=int(lis[2][1].split(' ')[0])
    res['Memory_Size']=int(lis[3][1].split(' ')[0])
    res['Max_Memory_Used']=int(lis[4][1].split(' ')[0])
    res['UTC_Timestamp'] = time.mktime(datetime.strptime(log[0]['value'], "%Y-%m-%d %H:%M:%S.%f").timetuple()) +timezone_offset*3600
    return res

### Prepare Logs

In [65]:
query_lambda_f1 = logclient.start_query(
        logGroupName='/aws/lambda/{}'.format("100ms"),
        queryString="fields @timestamp, @message| filter @message like 'REPORT'| sort @timestamp asc",
        startTime=app_exeuction_start_time,
        endTime=app_execution_end_time,
        limit=10000
        )
time.sleep(2)
query_lambda_f2 = logclient.start_query(
        logGroupName='/aws/lambda/{}'.format("200ms"),
        queryString="fields @timestamp, @message| filter @message like 'REPORT'| sort @timestamp asc",
        startTime=app_exeuction_start_time,
        endTime=app_execution_end_time,
        limit=10000
        )
time.sleep(2)
query_lambda_f3 = logclient.start_query(
        logGroupName='/aws/lambda/{}'.format("300ms"),
        queryString="fields @timestamp, @message| filter @message like 'REPORT'| sort @timestamp asc",
        startTime=app_exeuction_start_time,
        endTime=app_execution_end_time,
        limit=10000
        )
time.sleep(2)
query_lambda_f4 = logclient.start_query(
        logGroupName='/aws/lambda/{}'.format("400ms"),
        queryString="fields @timestamp, @message| filter @message like 'REPORT'| sort @timestamp asc",
        startTime=app_exeuction_start_time,
        endTime=app_execution_end_time,
        limit=10000
        )
time.sleep(2)

### Retrieve Logs

In [66]:
query_lambda_results_f1 = logclient.get_query_results(
        queryId=query_lambda_f1['queryId']
    )
time.sleep(1)
query_lambda_results_f2 = logclient.get_query_results(
        queryId=query_lambda_f2['queryId']
    )
time.sleep(1)
query_lambda_results_f3 = logclient.get_query_results(
        queryId=query_lambda_f3['queryId']
    )
time.sleep(1)
query_lambda_results_f4 = logclient.get_query_results(
        queryId=query_lambda_f4['queryId']
    )
time.sleep(1)

In [68]:
with open('query_lambda_results_f1.pickle', 'wb') as f:
    f.write(pickle.dumps(query_lambda_results_f1))
with open('query_lambda_results_f2.pickle', 'wb') as f:
    f.write(pickle.dumps(query_lambda_results_f2))
with open('query_lambda_results_f3.pickle', 'wb') as f:
    f.write(pickle.dumps(query_lambda_results_f3))
with open('query_lambda_results_f4.pickle', 'wb') as f:
    f.write(pickle.dumps(query_lambda_results_f4))

In [69]:
AppSFNDelay_duration_lambda_f1 = [lambda_report_log_to_dict(log) for log in query_lambda_results_f1['results']]
AppSFNDelay_duration_lambda_f2 = [lambda_report_log_to_dict(log) for log in query_lambda_results_f2['results']]
AppSFNDelay_duration_lambda_f3 = [lambda_report_log_to_dict(log) for log in query_lambda_results_f3['results']]
AppSFNDelay_duration_lambda_f4 = [lambda_report_log_to_dict(log) for log in query_lambda_results_f4['results']]

In [71]:
AppSFNDelay_duration_lambda_f1 = pd.DataFrame(AppSFNDelay_duration_lambda_f1)
AppSFNDelay_duration_lambda_f2 = pd.DataFrame(AppSFNDelay_duration_lambda_f2)
AppSFNDelay_duration_lambda_f3 = pd.DataFrame(AppSFNDelay_duration_lambda_f3)
AppSFNDelay_duration_lambda_f4 = pd.DataFrame(AppSFNDelay_duration_lambda_f4)

# Analyze the Delay

## Response time reported by Step Functions

In [57]:
AppSFNDelay_duration = pd.DataFrame((AppSFNDelay_sfn_logs['End'] - AppSFNDelay_sfn_logs['Start'])*1000, columns=['Duration'])
AppSFNDelay_duration.to_csv('AppSFNDelay_duration.csv', index=True)
AppSFNDelay_duration = pd.read_csv('AppSFNDelay_duration.csv', low_memory=False, index_col=0)

In [133]:
AppSFNDelay_duration_f1 = pd.DataFrame((AppSFNDelay_sfn_f1_logs['End'] - AppSFNDelay_sfn_f1_logs['Start'])*1000, columns=['Duration'])
AppSFNDelay_duration_f2 = pd.DataFrame((AppSFNDelay_sfn_f2_logs['End'] - AppSFNDelay_sfn_f2_logs['Start'])*1000, columns=['Duration'])
AppSFNDelay_duration_f3 = pd.DataFrame((AppSFNDelay_sfn_f3_logs['End'] - AppSFNDelay_sfn_f3_logs['Start'])*1000, columns=['Duration'])
AppSFNDelay_duration_f4 = pd.DataFrame((AppSFNDelay_sfn_f4_logs['End'] - AppSFNDelay_sfn_f4_logs['Start'])*1000, columns=['Duration'])
AppSFNDelay_duration_f1.to_csv('AppSFNDelay_duration_f1.csv', index=True)
AppSFNDelay_duration_f1 = pd.read_csv('AppSFNDelay_duration_f1.csv', low_memory=False, index_col=0)
AppSFNDelay_duration_f2.to_csv('AppSFNDelay_duration_f2.csv', index=True)
AppSFNDelay_duration_f2 = pd.read_csv('AppSFNDelay_duration_f2.csv', low_memory=False, index_col=0)
AppSFNDelay_duration_f3.to_csv('AppSFNDelay_duration_f3.csv', index=True)
AppSFNDelay_duration_f3 = pd.read_csv('AppSFNDelay_duration_f3.csv', low_memory=False, index_col=0)
AppSFNDelay_duration_f4.to_csv('AppSFNDelay_duration_f4.csv', index=True)
AppSFNDelay_duration_f4 = pd.read_csv('AppSFNDelay_duration_f4.csv', low_memory=False, index_col=0)

In [134]:
AppSFNDelay_scheduling_overhead_f1 = pd.DataFrame((AppSFNDelay_sfn_f1_logs['Start'] - AppSFNDelay_sfn_f1_logs['Scheduled'])*1000, columns=['scheduling_overhead'])
AppSFNDelay_scheduling_overhead_f2 = pd.DataFrame((AppSFNDelay_sfn_f2_logs['Start'] - AppSFNDelay_sfn_f2_logs['Scheduled'])*1000, columns=['scheduling_overhead'])
AppSFNDelay_scheduling_overhead_f3 = pd.DataFrame((AppSFNDelay_sfn_f3_logs['Start'] - AppSFNDelay_sfn_f3_logs['Scheduled'])*1000, columns=['scheduling_overhead'])
AppSFNDelay_scheduling_overhead_f4 = pd.DataFrame((AppSFNDelay_sfn_f4_logs['Start'] - AppSFNDelay_sfn_f4_logs['Scheduled'])*1000, columns=['scheduling_overhead'])
AppSFNDelay_scheduling_overhead_f1.to_csv('AppSFNDelay_scheduling_overhead_f1.csv', index=True)
AppSFNDelay_scheduling_overhead_f1 = pd.read_csv('AppSFNDelay_scheduling_overhead_f1.csv', low_memory=False, index_col=0)
AppSFNDelay_scheduling_overhead_f2.to_csv('AppSFNDelay_scheduling_overhead_f2.csv', index=True)
AppSFNDelay_scheduling_overhead_f2 = pd.read_csv('AppSFNDelay_scheduling_overhead_f2.csv', low_memory=False, index_col=0)
AppSFNDelay_scheduling_overhead_f3.to_csv('AppSFNDelay_scheduling_overhead_f3.csv', index=True)
AppSFNDelay_scheduling_overhead_f3 = pd.read_csv('AppSFNDelay_scheduling_overhead_f3.csv', low_memory=False, index_col=0)
AppSFNDelay_scheduling_overhead_f4.to_csv('AppSFNDelay_scheduling_overhead_f4.csv', index=True)
AppSFNDelay_scheduling_overhead_f4 = pd.read_csv('AppSFNDelay_scheduling_overhead_f4.csv', low_memory=False, index_col=0)

In [135]:
print('Number of Executions: ', len(AppSFNDelay_duration['Duration']))
AppSFNDelay_avg_duration_aws = np.mean(AppSFNDelay_duration['Duration'])
AppSFNDelay_mid_duration_aws = np.median(AppSFNDelay_duration['Duration'])
print('Average Duration Reported by AWS: ', AppSFNDelay_avg_duration_aws, 'ms')
print('Median Duration Reported by AWS: ', AppSFNDelay_mid_duration_aws, 'ms')
print('Standard Deviation of Duration Reported by AWS: ', np.std(AppSFNDelay_duration['Duration']), 'ms')

Number of Executions:  5000
Average Duration Reported by AWS:  1088.2806000232697 ms
Median Duration Reported by AWS:  1082.9999446868896 ms
Standard Deviation of Duration Reported by AWS:  19.523459986372146 ms


In [169]:
print(AppSFNDelay_scheduling_overhead_f1.mean())
print(AppSFNDelay_scheduling_overhead_f1.median())

scheduling_overhead    7.9234
dtype: float64
scheduling_overhead    0.0
dtype: float64


* Save the Scheduling Overhead Data

In [172]:
AppSFNDelay_scheduling_overhead_f1.to_csv("Scheduling_Overhead.csv", index=False)

* Other functions do not incur scheduling overhead

In [168]:
print(AppSFNDelay_scheduling_overhead_f2.mean())
print(AppSFNDelay_scheduling_overhead_f2.median())
print(AppSFNDelay_scheduling_overhead_f2.std())

scheduling_overhead    0.0228
dtype: float64
scheduling_overhead    0.0
dtype: float64
scheduling_overhead    0.14928
dtype: float64


In [138]:
print(AppSFNDelay_scheduling_overhead_f3.mean())
print(AppSFNDelay_scheduling_overhead_f3.median())

scheduling_overhead    0.0276
dtype: float64
scheduling_overhead    0.0
dtype: float64


In [139]:
print(AppSFNDelay_scheduling_overhead_f4.mean())
print(AppSFNDelay_scheduling_overhead_f4.median())

scheduling_overhead    0.024
dtype: float64
scheduling_overhead    0.0
dtype: float64


## Response time reported by Lambda

In [140]:
AppSFNDelay_duration_lambda_f1['Duration'] + AppSFNDelay_duration_lambda_f2['Duration'] + AppSFNDelay_duration_lambda_f3['Duration'] + AppSFNDelay_duration_lambda_f4['Duration']

0       1007.89
1       1007.96
2       1007.65
3       1007.72
4       1007.68
         ...   
4995    1007.97
4996    1007.89
4997    1007.71
4998    1007.99
4999    1007.77
Name: Duration, Length: 5000, dtype: float64

## Delay

In [90]:
delay = AppSFNDelay_duration.squeeze() - (AppSFNDelay_duration_lambda_f1['Duration'] + AppSFNDelay_duration_lambda_f2['Duration'] + AppSFNDelay_duration_lambda_f3['Duration'] + AppSFNDelay_duration_lambda_f4['Duration'])

In [91]:
delay.mean() / 4

20.04990800581741

In [92]:
delay.median() /4

18.78498617172241

## f1

In [155]:
print('mean:', (AppSFNDelay_duration_f1.squeeze() - AppSFNDelay_duration_lambda_f1['Duration']).mean())
print('median:', (AppSFNDelay_duration_f1.squeeze() - AppSFNDelay_duration_lambda_f1['Duration']).median())
print('avg:', ((AppSFNDelay_sfn_f1_logs['End'] - AppSFNDelay_sfn_f1_logs['Scheduled'])*1000 - AppSFNDelay_duration_lambda_f1['Duration']).mean())
print('median:', ((AppSFNDelay_sfn_f1_logs['End'] - AppSFNDelay_sfn_f1_logs['Scheduled'])*1000 - AppSFNDelay_duration_lambda_f1['Duration']).median())

mean: 21.67099466757202
median: 21.145025749206553
avg: 29.59439444975281
median: 22.250072479248047


## f2

In [156]:
print('mean:', (AppSFNDelay_duration_f2.squeeze() - AppSFNDelay_duration_lambda_f2['Duration']).mean())
print('median:', (AppSFNDelay_duration_f2.squeeze() - AppSFNDelay_duration_lambda_f2['Duration']).median())
print('avg:', ((AppSFNDelay_sfn_f2_logs['End'] - AppSFNDelay_sfn_f2_logs['Scheduled'])*1000 - AppSFNDelay_duration_lambda_f2['Duration']).mean())
print('median:', ((AppSFNDelay_sfn_f2_logs['End'] - AppSFNDelay_sfn_f2_logs['Scheduled'])*1000 - AppSFNDelay_duration_lambda_f2['Duration']).median())

mean: 15.76608007781983
median: 14.160080108642546
avg: 15.78887999885559
median: 14.170080108642566


## f3

In [157]:
print('mean:', (AppSFNDelay_duration_f3.squeeze() - AppSFNDelay_duration_lambda_f3['Duration']).mean())
print('median:', (AppSFNDelay_duration_f3.squeeze() - AppSFNDelay_duration_lambda_f3['Duration']).median())
print('avg:', ((AppSFNDelay_sfn_f3_logs['End'] - AppSFNDelay_sfn_f3_logs['Scheduled'])*1000 - AppSFNDelay_duration_lambda_f3['Duration']).mean())
print('median:', ((AppSFNDelay_sfn_f3_logs['End'] - AppSFNDelay_sfn_f3_logs['Scheduled'])*1000 - AppSFNDelay_duration_lambda_f3['Duration']).median())

mean: 14.454029137542728
median: 13.909984741210963
avg: 14.48162904446411
median: 13.919984741210953


## f4

In [158]:
print('mean:', (AppSFNDelay_duration_f4.squeeze() - AppSFNDelay_duration_lambda_f4['Duration']).mean())
print('median:', (AppSFNDelay_duration_f4.squeeze() - AppSFNDelay_duration_lambda_f4['Duration']).median())
print('avg:', ((AppSFNDelay_sfn_f4_logs['End'] - AppSFNDelay_sfn_f4_logs['Scheduled'])*1000 - AppSFNDelay_duration_lambda_f4['Duration']).mean())
print('median:', ((AppSFNDelay_sfn_f4_logs['End'] - AppSFNDelay_sfn_f4_logs['Scheduled'])*1000 - AppSFNDelay_duration_lambda_f4['Duration']).median())

mean: 14.3305294544983
median: 13.860127792358412
avg: 14.35452962234497
median: 13.869889373779301


* Save the Function Execution Delay Data

In [175]:
f2_delay = AppSFNDelay_duration_f2.squeeze() - AppSFNDelay_duration_lambda_f2['Duration']
f3_delay = AppSFNDelay_duration_f3.squeeze() - AppSFNDelay_duration_lambda_f3['Duration']
f4_delay = AppSFNDelay_duration_f4.squeeze() - AppSFNDelay_duration_lambda_f4['Duration']

In [179]:
function_execution_delay = f2_delay.append(f3_delay).append(f4_delay)

In [184]:
function_execution_delay.to_csv("Function_Execution_Delay.csv", index=False)